# Lesson 19: Neural Networks with Keras

## Section 19.0: Import Packages and Load Data

In [ ]:
from __future__ import print_function
import random
import numpy as np
import pandas as pd
from math import sin
import seaborn as sns

# Use: conda install keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD   # Stochastic Gradient Descent

from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import sklearn.cross_validation as cv
import sklearn.datasets as datasets
from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (8, 8)

In [ ]:
# Create some data (noisy cubic polynomial)
def f(x):
    return x ** 3 - 5 * x + 12 + random.random()

X = np.linspace(-1, 1, 1000).reshape(-1, 1)
y = np.array(list(map(f, X)))

print(X.shape, y.shape)

## Section 19.1: Neural Network Regression

### Baseline with Linear Regression model
Let's build a baseline model to compare our NN results with.

In [ ]:
lm = linear_model.LinearRegression()
lm.fit(X,y)
linear_predictions = lm.predict(X)

### Polynomial Regression
Let's train a neural network on a few different shapes. First we start with a polynomial (a cubic).

In [ ]:
# Define a Feed Forward NN
model = Sequential()
model.add(Dense(input_dim=1, units=5))
model.add(Activation('tanh'))
model.add(Dense(input_dim=5, units=1))
model.add(Activation('linear'))

#   lr: learning rate
model.compile(loss='mse', optimizer=SGD(lr=0.01))

In [ ]:
# Train the model
print('Training...')
loss = model.fit(X, y, epochs=500, validation_split=0.1, 
                 batch_size=128, verbose=False)
print(loss.history['loss'][-1])
print("Done")

In [ ]:
# Plot the predictions
predictions = model.predict(X)

plt.scatter(X, y)
plt.plot(X, linear_predictions, color='y') # plot baseline model predictions
plt.plot(X, predictions, color='r') # plot nn model predictions
plt.show()
print("MSE", mean_squared_error(predictions, y))

### Sine Regression
Now, let's train our network on a sinusoid.

In [ ]:
# Sine data
X = np.linspace(0, 2 * np.pi, 500).reshape(-1,1)
y = np.sin(X)

print(X.shape, y.shape)

In [ ]:
# Create and train baseline (linear regression) model
lm = linear_model.LinearRegression()
lm.fit(X,y)
linear_predictions = lm.predict(X)

# Create the neural net model
model = Sequential()
model.add(Dense(input_dim=1, units=5))
model.add(Activation('tanh'))
model.add(Dense(input_dim=5, units=1))
model.add(Activation('linear'))

# Set lr: learning rate
model.compile(loss='mse', optimizer=SGD(lr=0.1))

In [ ]:
# Train the model
print('Training..')
loss = model.fit(X, y, epochs=150, validation_split=0.1, 
                 batch_size=128, verbose=False)
print(loss.history['loss'][-1])
print('Complete')

In [ ]:
# Get nn model predictions
predictions = model.predict(X)

# Plot the predictions
plt.scatter(X, y)
plt.plot(X, linear_predictions, color='y')
plt.plot(X, predictions, color='r')
plt.show()
print("MSE", mean_squared_error(predictions, y))

In [ ]:
# Plot the error over time
plt.scatter(range(len(loss.history['loss'])), loss.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('MSE by Epoch');

### Train longer

If we train for more epochs, we can get a better regression.

In [ ]:
X = np.linspace(0, 2 * np.pi, 1000).reshape(-1,1)
y = np.sin(X)

print(X.shape, y.shape)

model = Sequential()
model.add(Dense(input_dim=1, units=5))
model.add(Activation('tanh'))
model.add(Dense(input_dim=5, units=1))
model.add(Activation('linear'))

#   lr: learning rate
model.compile(loss='mse', optimizer=SGD(lr=0.05))

print('Training..')
loss = model.fit(X, y, epochs=15000, validation_split=0.1, 
                 batch_size=128, verbose=False)
print(loss.history['loss'][-1])
print('Complete')

# Plot
predictions = model.predict(X)

plt.scatter(X, y)
plt.plot(X, predictions, color='r')
plt.show()
print("MSE", mean_squared_error(predictions, y))

We can take a closer look at the error per training epoch.

In [ ]:
# Plot the error over time
plt.scatter(range(len(loss.history['loss'])), loss.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('MSE by Epoch')

### Exercise 19.2: Perform regression on the following data
Hints:
* Use examples from above
* Try adding a hidden layer
* Try lowering the learning rate and using more epochs

In [ ]:
def f(x):
    return x ** 2 * np.sin(x**2)

# Sine data
X = np.linspace(2, np.pi, 1000).reshape(-1,1)
y = np.array(list(map(f, X)))

print(X.shape, y.shape)

In [ ]:
## Add model here




## Fit model here




# Get models predictions
predictions = model.predict(X)

# Plot
plt.scatter(X, y)
plt.plot(X, predictions, color='r')
plt.show()
print("MSE", mean_squared_error(predictions, y))

## Section 19.3: Classification

We'll start with the Iris dataset (of course).

In [ ]:
iris = datasets.load_iris()

X = iris.data
y = iris.target

# Break each output into indicator cols
y_cat = pd.get_dummies(y).values

print(X.shape, y_cat.shape)

In [ ]:
# Define a model
model = Sequential()

# input_dim = number of neurons in previous layer
# output_dim = number of neurons in current layer

# First layer - input_dim=k features
model.add(Dense(input_dim=4, units=4))
model.add(Activation("tanh"))

model.add(Dense(input_dim=4, units=4))
model.add(Activation("tanh"))


# Output layer - output_dim=# of output per point (in y)
# Use 'softmax' for class probability or 'linear' for regression
model.add(Dense(input_dim=4, units=3))
model.add(Activation("softmax"))

# Uses Mean Squared Error and Stochastic Gradient Descent
model.compile(loss='mse', optimizer=SGD(lr=0.01))

In [ ]:
# Train the model
print('Training...')
loss = model.fit(X, y_cat, 
                 validation_split=0.1, epochs=5000, 
                 batch_size=16, verbose=False)

print(loss.history['loss'][-1])   # displays MSE at last iteration
print("Training complete")

In [ ]:
# Model evaluation
pred_y = model.predict(X, verbose=False)
preds  = model.predict_classes(X, verbose=False)

# Show prediction example
print('TRAINING SAMPLE: ' + str(X[0]))
print('PREDICTION: ' + iris.target_names[preds[0]])
print('TRUTH: ' + iris.target_names[y[0]])
print('')
print('ACCURACY: ', accuracy_score(y, preds))
print('CONFUSION MATRIX:\n', confusion_matrix(y, preds))

In [ ]:
# Plot the error over time
plt.scatter(range(len(loss.history['loss'])), loss.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('MSE by Epoch')

### Abalone data set

In [ ]:
columns = ["Sex", "Length", "Diameter", "Height", "Whole Weight",
           "Shucked weight", "Viscera weight", "Shell weight", "Rings" ]
df = pd.read_csv("./datasets/abalone.data", names=columns)

df.head()

In [ ]:
sns.pairplot(data=df, vars=columns[1:], hue="Sex")
plt.show()

In [ ]:
d = {'M': 0, 'F': 1, 'I': 2}
invert_d = {v: k for k, v in d.items()}
df["Sex"] = df["Sex"].apply(lambda x: d[x])

In [ ]:
X = np.array(df[columns[1:]])
y = np.array(df["Sex"])
y_cat = pd.get_dummies(y).values

print(X.shape, y_cat.shape)

In [ ]:
# Define a model
model = Sequential()

# input_dim = number of neurons in previous layer
# output_dim = number of neurons in current layer

# First layer - input_dim=k features
model.add(Dense(input_dim=8, units=6))
model.add(Activation("tanh"))

model.add(Dense(input_dim=6, units=6))
model.add(Activation("tanh"))

# Output layer - output_dim=# of output per point (in y)
# Use 'softmax' for class probability or 'linear' for regression
model.add(Dense(input_dim=6, units=3))
model.add(Activation("softmax"))

# Uses Mean Squared Error and Stochastic Gradient Descent
model.compile(loss='mse', optimizer=SGD(lr=0.1))

In [ ]:
# Train the model
print('Training...')
loss = model.fit(X, y_cat, 
                 validation_split=0.1, epochs=1000, 
                 batch_size=16, verbose=False)

print(loss.history['loss'][-1]) # displays MSE at last iteration
print("Training complete")

In [ ]:
# Model evaluation
pred_y = model.predict(X, verbose=False)
preds  = model.predict_classes(X, verbose=False)

# Show prediction example
print('TRAINING SAMPLE: ' + str(X[0]))
print('PREDICTION: ' + invert_d[preds[0]])
print('TRUTH: ' + invert_d[y[0]])
print('')
print('ACCURACY: ', accuracy_score(y, preds))
print('CONFUSION MATRIX:\n', confusion_matrix(y, preds))

# Plot the error over time
plt.scatter(range(len(loss.history['loss'])), loss.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('MSE by Epoch')

### Exercise 19.4: Perform classification on the following data

Classify the following data ([source](https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope)). You'll need to translate the classes into integers and make dummies. Design a neural network to classify the data and evaluate the results.

In [ ]:
names = "fLength fWidth fSize fConc fConc1 fAsym fM3Long fM3Trans fAlpha fDist class".split()
df = pd.read_csv("./datasets/magic04.data", names=names)
df.head()

In [ ]:
d = {'g': 0, 'h': 1}
invert_d = {v: k for k, v in d.items()}
df["class"] = df["class"].apply(lambda x: d[x])

In [ ]:
X = np.array(df[df.columns[:-1]])
y = np.array(df["class"])
y_cat = pd.get_dummies(y).values

print(X.shape, y_cat.shape)

In [ ]:
## Define model here


In [ ]:
## Train model here


In [ ]:
## Evaluate model here
